In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import ast

In [ ]:
# 런타임 오류 방지 함수
# 이 함수는 Colab 연결을 유지하기 위해 60초마다 연결 버튼을 자동으로 클릭
def keep_alive():
    display(Javascript('''
        function ClickConnect(){
            console.log("클릭 연결 버튼");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 60000)
    '''))


In [ ]:
# 이 클래스는 데이터를 불러오고 처리하는 방법을 정의
# img_dir은 이미지가 저장된 폴더 경로, df는 데이터프레임, facepart는 얼굴 부위 번호
# 데이터셋 클래스 정의
class CachedDataset(Dataset):
    def __init__(self, img_dir, df, facepart, task, transform=None, cache_dir='/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/cache'):
        self.transform = transform
        self.facepart = facepart
        self.task = task
        os.makedirs(cache_dir, exist_ok=True)
        self.cache_file = os.path.join(cache_dir, f'cache_facepart_{facepart}_{task}.pkl')

        if os.path.exists(self.cache_file):
            print(f"facepart {facepart}의 {task} 캐시된 데이터를 불러옵니다...")
            with open(self.cache_file, 'rb') as f:
                self.cache = pickle.load(f)
        else:
            print(f"facepart {facepart}의 {task} 캐시를 생성합니다...")
            self.cache = []
            df_facepart = df[df['images'].apply(lambda x: x['facepart'] in ([facepart] if facepart not in [3, 4, 5, 6] else [3, 4] if facepart == 3 else [5, 6]))]

            for idx, row in df_facepart.iterrows():
                try:
                    bbox = row['images']['bbox']
                    if (bbox == ['None', 'None', 'None', 'None']) or not all(isinstance(b, (int, float)) and b is not None for b in bbox):
                        continue

                    img_name = row['info']['filename']
                    if facepart == 0:
                        img_path = os.path.join(img_dir, img_name)
                    else:
                        img_path = os.path.join(img_dir, f"{os.path.splitext(img_name)[0]}_{facepart}.jpg")

                    if not os.path.exists(img_path):
                        continue

                    try:
                        image = Image.open(img_path).convert('RGB')
                    except (IOError, OSError):
                        continue

                    if self.transform:
                        image = self.transform(image)

                    labels = self._prepare_labels(row['annotations'], row['equipment'], row['info'])
                    if labels:
                        self.cache.append((image, torch.tensor(labels, dtype=torch.float if task == 'regression' else torch.long)))

                except Exception as e:
                    print(f"데이터 처리 중 오류 발생: {str(e)}")
                    continue

            with open(self.cache_file, 'wb') as f:
                pickle.dump(self.cache, f)
            print(f"facepart {facepart}의 {task} 캐시가 생성되고 저장되었습니다")

    def _prepare_labels(self, annotations, equipment, info):
        labels = []
        try:
            if self.task == 'classification':
                if self.facepart == 0:
                    labels = [info['skin_type'], info['sensitive']]
                if annotations:
                    labels.extend(self._process_annotations(annotations))
            elif self.task == 'regression':
                if equipment:
                    labels.extend(list(equipment.values()))
        except Exception as e:
            print(f"레이블 준비 중 오류 발생: {str(e)}")
        return labels

    def _process_annotations(self, annotations):
        processed = []
        for key, value in annotations.items():
            if isinstance(value, (int, float)):
                processed.append(value)
        return processed

    def __len__(self):
        return len(self.cache)

    def __getitem__(self, idx):
        return self.cache[idx]


In [ ]:
# ResNet50 모델 생성 함수
# 사전 학습된 ResNet50 모델을 로드, 마지막 fully connected 층을 num_outputs에 맞게 수정
# dropout 층을 추가하여 과적합을 방지
def create_resnet_model(num_outputs, task):
    model = models.resnet50(pretrained=True)
    num_features = model.fc.in_features
    if task == 'classification':
        model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, num_outputs)
        )
    else:  # regression
        model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, num_outputs),
            nn.ReLU() # ReLU를 사용하여 음수 값 방지
        )
    return model

# 현재 iteration(현재 모델이 몇 번째 반복을 수행 중인지)에 따라 학습률을 감소
# 학습이 진행됨에 따라 학습률을 점진적으로 줄여나가는 역할
# 학습 초기에는 큰 학습률로 빠르게 학습하다가, 학습이 진행될수록 작은 학습률로 세밀하게 조정
# 모델이 더 안정적으로 수렴하도록 도움
def lr_poly(base_lr, iter, max_iter, power):
    return base_lr * ((1 - float(iter) / max_iter) ** power)


In [ ]:
# 모델을 학습, 검증, 체크포인트 저장, 학습 과정 시각화
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, facepart, task):
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    train_metrics = []
    val_metrics = []
    save_path = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model'

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_metric = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # 학습률 조정
            lr = lr_poly(1e-3, epoch * len(train_loader) + i, num_epochs * len(train_loader), 0.9)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

            running_loss += loss.item()

            # 평가 지표 계산
            if task == 'classification':
                _, predicted = torch.max(outputs.data, 1)
                running_metric += (predicted == labels).sum().item() / labels.size(0)
            else:  # regression
                running_metric += nn.L1Loss()(outputs, labels).item()

        epoch_loss = running_loss / len(train_loader)
        epoch_metric = running_metric / len(train_loader)
        train_losses.append(epoch_loss)
        train_metrics.append(epoch_metric)
        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, {'Accuracy' if task == 'classification' else 'MAE'}: {epoch_metric:.4f}")

        # 검증
        model.eval()
        val_loss = 0.0
        val_metric = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # 평가 지표 계산
                if task == 'classification':
                    _, predicted = torch.max(outputs.data, 1)
                    val_metric += (predicted == labels).sum().item() / labels.size(0)
                else:  # regression
                    val_metric += nn.L1Loss()(outputs, labels).item()

        val_loss /= len(val_loader)
        val_metric /= len(val_loader)
        val_losses.append(val_loss)
        val_metrics.append(val_metric)
        print(f"Validation Loss: {val_loss:.4f}, {'Accuracy' if task == 'classification' else 'MAE'}: {val_metric:.4f}")

        # 최고의 모델 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), os.path.join(save_path, f'best_model_resnet50_facepart_{facepart}_{task}.pth'))
            print(f"facepart {facepart}의 {task} 새로운 최고 모델을 저장했습니다")

        # 체크포인트 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
        }, os.path.join(save_path, f'checkpoint_resnet50_facepart_{facepart}_{task}_epoch_{epoch+1}.pth'))

    # 학습 과정 시각화 및 저장
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
    plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss for ResNet50 Facepart {facepart} {task}')
    plt.legend()
    plt.xticks(range(0, num_epochs+1, 5))

    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs+1), train_metrics, label='Train Metric')
    plt.plot(range(1, num_epochs+1), val_metrics, label='Validation Metric')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy' if task == 'classification' else 'MAE')
    plt.title(f'Training and Validation Metric for ResNet50 Facepart {facepart} {task}')
    plt.legend()
    plt.xticks(range(0, num_epochs+1, 5))

    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f'plot_resnet50_facepart_{facepart}_{task}.png'))
    plt.close()

    return model

In [ ]:
# 메인함수
def main(facepart_range):
    base_path = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터'

    try:
        df = pd.read_csv(os.path.join(base_path, 'json to df.csv'))
    except FileNotFoundError:
        print("CSV 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        return
    except pd.errors.EmptyDataError:
        print("CSV 파일이 비어있습니다.")
        return
    except pd.errors.ParserError:
        print("CSV 파일 파싱 중 오류가 발생했습니다. 파일 형식을 확인해주세요.")
        return

    # 문자열 딕셔너리를 실제 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

    # Training 데이터만 선택
    df_train = df[df['split'] == 'Training']

    # 이미지 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for facepart in facepart_range:
        for task in ['classification', 'regression']:
            if task == 'regression' and facepart in [2, 7]:
                continue

            print(f"facepart {facepart} {task} 처리 중")

            if facepart == 0:
                img_dir = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training/01.원천데이터'
            else:
                img_dir = f'/gdrive/MyDrive/Final project/1_Red/4_데이터탐색_전처리/facepart별 피부 이미지/Training_cropped/{facepart}'

            if not os.path.exists(img_dir):
                print(f"이미지 디렉토리를 찾을 수 없습니다: {img_dir}")
                continue

            # 데이터셋 생성
            try:
                dataset = CachedDataset(img_dir, df_train, facepart, task, transform)
            except Exception as e:
                print(f"데이터셋 생성 중 오류 발생: {str(e)}")
                continue

            # 데이터셋을 train과 validation으로 분할
            train_data, val_data = train_test_split(dataset, test_size=0.1, random_state=42)

            # 데이터 로더 생성
            train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=4)
            val_loader = DataLoader(val_data, batch_size=16, shuffle=False, num_workers=4)

            # 모델 생성
            num_outputs = len(dataset[0][1])
            model = create_resnet_model(num_outputs, task).to(device)

            # 손실 함수와 최적화 알고리즘 정의
            criterion = nn.CrossEntropyLoss() if task == 'classification' else nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0005)

            # ResNet50 모델 학습
            try:
                model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=30, device=device, facepart=facepart, task=task)
            except Exception as e:
                print(f"ResNet50 모델 학습 중 오류 발생: {str(e)}")

            # 최종 모델 저장
            save_path = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단'
            try:
                torch.save(model.state_dict(), os.path.join(save_path, f'final_model_resnet50_facepart_{facepart}_{task}.pth'))
            except Exception as e:
                print(f"모델 저장 중 오류 발생: {str(e)}")

# 1번 강다율 2번 문수정 3번 김길현

In [ ]:
# 메인 실행
if __name__ == "__main__":
    keep_alive() # 연결 유지 함수 실행
    # 사용자 입력을 받아 facepart 범위 설정
    user_input = input("처리할 facepart 범위를 선택하세요 (1: 0-2, 2: 3-6, 3: 7-8): ")
    if user_input == '1':
        facepart_range = range(0, 3)
    elif user_input == '2':
        facepart_range = range(3, 7)
    elif user_input == '3':
        facepart_range = range(7, 9)
    else:
        print("잘못된 입력입니다. 프로그램을 종료합니다.")
        exit()

    main(facepart_range)

# 코드 설명
- class CachedDataset(Dataset):
    - 이 클래스는 데이터셋을 생성하고 캐시합니다.
    - '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터'에서 이미지를 로드하고
    - '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/cache'에 캐시를 저장합니다.
    - 각 얼굴 부위(facepart)별로 별도의 캐시를 생성합니다.

- def create_resnet_model(num_outputs):
    - ResNet50 모델을 생성하고 마지막 레이어를 수정하여 원하는 출력 수에 맞춥니다.

- def lr_poly(base_lr, iter, max_iter, power):
    - 학습률을 동적으로 조정하는 함수입니다.

- def train_model(model, train_loader, val_loader, criterion, optimizer,num_epochs, device, facepart):
    - 모델을 훈련시키는 함수입니다.
    - 훈련 중 최고의 모델을 '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model'에 저장합니다.
    - 훈련 과정을 시각화하여 같은 경로에 저장합니다.

- def main():
    - 메인 함수로, 전체 훈련 과정을 제어합니다.
    - 1. CSV 파일을 로드하고 전처리합니다.
    - 2. 각 얼굴 부위(facepart)에 대해 반복:
    -    a. 데이터셋을 생성합니다.
    -    b. 데이터를 훈련 세트와 검증 세트로 나눕니다.
    -    c. 모델을 생성하고 훈련시킵니다.
    -    d. 최종 모델을 저장합니다.
